# references
1. https://www.kaggle.com/columbia2131/device-eda-interpolate-by-removing-device-en-ja

# Load Libraries

In [85]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [91]:
RANDOM_STATE = 1990
lr = 0.01
batch_size = 64
EPOCH_NUM = 1000

torch.manual_seed(RANDOM_STATE)

# Useful Functions

In [87]:
def get_groundtruth(path: Path) -> pd.DataFrame:
    output_df = pd.DataFrame()
    
    for path in glob(str(path / 'train/*/*/ground_truth.csv')):
        _df = pd.read_csv(path)
        output_df = pd.concat([output_df, _df])
    output_df = output_df.reset_index(drop=True)
    
    _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
    output_df[['t_'+col for col in _columns]] = output_df[_columns]
    output_df = output_df.drop(columns=_columns, axis=1)
    return output_df

In [88]:
def calc_haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    
    c = 2 * np.arcsin(a ** 0.5)
    dist = 6_367_000 * c
    return dist

In [89]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

In [90]:
def check_score_np(predict:np.array, target:np.array):
    m = []
    for i in range(predict.shape[0]):
        temp = calc_haversine(predict[i,0], predict[i,1], target[i,0], target[i,1])
        m.append(temp)
    
    m = np.array(m)
    score = (np.percentile(m, 50) + np.percentile(m, 95))/2
    
    return score

# Load Datasets

In [15]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")
train_df = pd.read_csv(data_dir / "baseline_locations_train.csv")
test_df = pd.read_csv(data_dir / "baseline_locations_test.csv")

# merge graoundtruth
train_df = train_df.merge(
    get_groundtruth(data_dir),
    on=['collectionName', 'phoneName', 'millisSinceGpsEpoch']
)

In [18]:
# check score
train_df = check_score(train_df)

error meter: 3.846848374990639
score: 5.287970649084159


# Feature Engineering

In [20]:
train_df.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,speedMps,courseDegree,t_latDeg,t_lngDeg,t_heightAboveWgs84EllipsoidM,meter
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423575,-122.094091,-34.06,2020-05-14-US-MTV-1_Pixel4,551.44,1.1,0.0,0.0,12.7,37.423576,-122.094132,33.21,3.586842
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423578,-122.094101,-33.29,2020-05-14-US-MTV-1_Pixel4,552.44,1.1,0.0,0.0,12.7,37.423576,-122.094132,33.21,2.745901
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423573,-122.094111,-30.99,2020-05-14-US-MTV-1_Pixel4,553.44,1.1,0.0,0.0,12.7,37.423576,-122.094132,33.21,1.888409
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423583,-122.094121,-32.83,2020-05-14-US-MTV-1_Pixel4,554.44,1.1,0.0,0.0,12.7,37.423576,-122.094132,33.20,1.213483
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423579,-122.094114,-34.49,2020-05-14-US-MTV-1_Pixel4,555.44,1.1,0.0,0.0,12.7,37.423576,-122.094132,33.20,1.650722


In [26]:
test_df.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040,-31.76,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416653,-122.082039,-31.65,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063,-31.52,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073,-28.95,2020-05-15-US-MTV-1_Pixel4


In [53]:
def extract_features(df, train = True):
    Xcols = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
    Ycols = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM']
    
    X = df[Xcols]
    if train:
        y = df[Ycols]
    else:
        y = None
    
    return np.array(X), np.array(y)

X, y = extract_features(train_df)
# Xtest, _ = extract_features(test_df, False)

print(X.shape, y.shape)

(131342, 3) (131342, 3)


# Modeling

In [54]:
# build model
class SimpleNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.fc1 = nn.Linear(input_size, 10)
        self.fc2 = nn.Linear(10, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x
    
model = SimpleNetwork(X.shape[1], y.shape[1])
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [56]:
# Create DataLoader
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/7, random_state = 1990)

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)

loader_train = DataLoader(ds_train, batch_size = batch_size, shuffle = True)
loader_test = DataLoader(ds_test, batch_size = batch_size, shuffle = False)

In [62]:
def train(epoch):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    for data, targets in loader_train:

        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정

    print("epoch{}：완료\n".format(epoch))

In [92]:
def test():
    model.eval()  # 신경망을 추론 모드로 전환
    correct = 0

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets in loader_test:

            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            predict.append(outputs)
            ground.append(targets)

    # 정확도 출력
    predict = np.concatenate(predict,axis = 0)
    ground = np.concatenate(ground,axis = 0)
    
    print(predict.shape, ground.shape)
    
    loss = loss_func(outputs, targets)
    meas = check_score_np(outputs, targets)
    print('\n테스트 데이터에서 예측 정확도: {}, {}\n'.format(loss, meas))

In [93]:
test()

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 13.305831909179688, 460101.300390625



In [ ]:
for epoch in range(EPOCH_NUM):
    train(epoch)
    test()


epoch0：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 7.282623291015625, 62519.51787109375

epoch1：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 20.24148178100586, 28313.8607421875

epoch2：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 12.859976768493652, 104701.8046875

epoch3：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 16.339576721191406, 88519.0908203125

epoch4：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 6.906274795532227, 83880.9701171875

epoch5：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 24.31685447692871, 19932.20341796875

epoch6：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 5.772268295288086, 182089.398828125

epoch7：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 60.89895248413086, 134938.24453125

epoch8：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 5.003030776977539, 28778.380957031248

epoch9：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 6.198967456817627, 60241.63056640625

epoch10：완료

(18764, 3) (18764, 3)

테스트 데이터에서 예측 정확도: 9.774504661560059, 53451.8244140625

epoch11：완료

(187

# Submission